<a href="https://colab.research.google.com/github/microprediction/endersnotebooks/blob/main/momentum_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade git+https://github.com/microprediction/endersgame.git
# It's probably fine to use the simpler import by the time your read this :)
#!pip install --upgrade endersgame

  Cloning https://github.com/microprediction/endersgame.git to /tmp/pip-req-build-o8gyvpvs
  Running command git clone --filter=blob:none --quiet https://github.com/microprediction/endersgame.git /tmp/pip-req-build-o8gyvpvs
  Resolved https://github.com/microprediction/endersgame.git to commit dffb0ab414907eb2e9885a38e51384a179b2bb2c
  Preparing metadata (setup.py) ... done


# Momentum Attacker
This notebook demonstrates how to create an `Attacker` described in [attacker.md](https://github.com/microprediction/endersgame/blob/main/endersgame/attackers/attacker.md). You may want to glance at this [notebook](https://github.com/microprediction/endersnotebooks/blob/main/mean_reversion_attacker.ipynb) also, if you seek more context or wish to know how these attackers can be used in a new tournament.



In [9]:
from endersgame import Attacker
from endersgame import stream_generator_generator   # <--- Provides training and test data
from endersgame.accounting.pnlutil import zero_pnl_summary, add_pnl_summaries # <-- Minor conveniences
from pprint import pprint
from endersgame import FEWMean, FEWVar  # <--- Utils to track running mean and var
import math

### Creating a Momentum based Attacker
We derive from `Attacker` and use the utilities `FEWMean` and `FEWVar` to track the running exponentially weighted quantities we need.

In [6]:

class MyAttacker(Attacker):

     def __init__(self, fast_fading_factor:dict=0.1, slow_fading_factor=0.01, diff_fading_factor=0.001, threshold = 2, burn_in=100, **kwargs):
         super().__init__(**kwargs)
         self.threshold = threshold
         self.fast_ewa = FEWMean(fading_factor=fast_fading_factor)   # <--- Track fast expon-weighted moving average
         self.slow_ewa = FEWMean(fading_factor=slow_fading_factor)   # <--- Track slow expon-weighted moving average
         self.diff_var = FEWVar(fading_factor=diff_fading_factor)    # <--- Tracks mean and var of the difference between the two
         self.countdown = burn_in

     def tick(self, x:float):
         self.fast_ewa.tick(x=x)    # <--- Update the fast expon avg
         self.slow_ewa.tick(x=x)    # <--- Update the slow expon avg
         fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
         self.diff_var.tick(x=fast_minus_slow)  # <--- Update var of diff
         self.countdown -= 1        # <--- Soon we'll be warm

     def predict(self, horizon: int = None) -> float:
         """
               We buy if signal > threshold*(trailing std of signal)
         """
         if self.countdown>0:
             return 0    # Not warmed up
         fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
         try:
             fast_minus_slow_std = math.sqrt( self.diff_var.get())
             decision = int(fast_minus_slow/(self.threshold*fast_minus_slow_std))  # <--- Create a buy (>0) or sell (<0) decision
             return decision
         except ArithmeticError:
             return 0

## Run the attacker on mock data
We use `tick_and_predict` from the parent class as this will track profit and loss for us.

In [7]:
attacker = MyAttacker()               # Always reset an attacker

xs = [1,3,4,2,4,5,1,5,2,5,10]*100
for x in xs:
   y = attacker.tick_and_predict(x=x)

## Run the attacker on real data
We reset the attacker every time it encounters a new stream, but track aggregate statistics.

In [13]:
gen_gen = stream_generator_generator(category='train')    # <-- You might want to change 'train' to 'test'
attacker = MyAttacker()
total_pnl = zero_pnl_summary()
for stream in gen_gen:
    for message in stream:
        attacker.tick_and_predict(x=message['x'])
    stream_pnl = attacker.pnl.summary()
    total_pnl = add_pnl_summaries(total_pnl,stream_pnl)

total_pnl.update({'profit_per_decision':total_pnl['total_profit']/total_pnl['num_resolved_decisions']})
pprint(total_pnl)

{'current_ndx': 19871117,
 'losses': 7842,
 'num_resolved_decisions': 14562,
 'profit_per_decision': 0.07927460391195962,
 'total_profit': 1154.396782165956,
 'wins': 6720}


And that's all we have. Again, you may want to refer to this [notebook](https://github.com/microprediction/endersnotebooks/blob/main/mean_reversion_attacker.ipynb) also.